In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import argparse
import secrets
import json
import sys
import math 

In [3]:
from patient.simulator import run_multi_seed
from patient.baseline_policies import *
from patient.lp_policies import *
from patient.group_based_policies import *
from patient.ordering_policies import *
from patient.provider_policies import *
from patient.utils import get_save_path, delete_duplicate_results, restrict_resources, one_shot_policy, MyEncoder

In [4]:
is_jupyter = 'ipykernel' in sys.modules

In [60]:
if is_jupyter: 
    seed        = 45
    num_patients = 25
    num_providers = 25
    provider_capacity = 1
    top_choice_prob = 1
    true_top_choice_prob = 1
    choice_model = "uniform_choice"
    exit_option = 0.5
    utility_function = "normal"
    out_folder = "policy_comparison"
    num_repetitions = 1
    num_trials = 100
    context_dim = 5
    max_menu_size = 25
    previous_patients_per_provider = 10
    order="custom"
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--seed', help='Random Seed', type=int, default=42)
    parser.add_argument('--n_patients',         '-N', help='Number of patients', type=int, default=100)
    parser.add_argument('--n_providers',        help='Number of providers', type=int, default=100)
    parser.add_argument('--n_trials',          help='Number of trials ', type=int, default=2)
    parser.add_argument('--top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--true_top_choice_prob',          help='Probability of picking top choice', type=float, default=0.75)
    parser.add_argument('--context_dim',          help='Context dim for patients and providers', type=int, default=5)
    parser.add_argument('--max_menu_size',          help='Context dim for patients and providers', type=int, default=50)
    parser.add_argument('--num_repetitions',          help='Context dim for patients and providers', type=int, default=1)
    parser.add_argument('--previous_patients_per_provider',          help='Context dim for patients and providers', type=int, default=10)
    parser.add_argument('--provider_capacity', help='Provider Capacity', type=int, default=5)
    parser.add_argument('--choice_model', help='Which choice model for patients', type=str, default='uniform_choice')
    parser.add_argument('--exit_option', help='What is the value of the exit option', type=float, default=0.5)
    parser.add_argument('--out_folder', help='Which folder to write results to', type=str, default='policy_comparison')
    parser.add_argument('--utility_function', help='Which folder to write results to', type=str, default='uniform')
    parser.add_argument('--order', help='Which folder to write results to', type=str, default='random')

    args = parser.parse_args()

    seed = args.seed
    num_patients = args.n_patients
    num_providers = args.n_providers 
    provider_capacity = args.provider_capacity
    top_choice_prob = args.top_choice_prob
    choice_model = args.choice_model
    exit_option = args.exit_option
    out_folder = args.out_folder
    num_trials = args.n_trials 
    context_dim = args.context_dim 
    num_repetitions = args.num_repetitions
    true_top_choice_prob = args.true_top_choice_prob
    max_menu_size = args.max_menu_size
    utility_function = args.utility_function
    order = args.order
    previous_patients_per_provider = args.previous_patients_per_provider

save_name = secrets.token_hex(4)  

In [61]:
results = {}
results['parameters'] = {'seed'      : seed,
        'num_patients'    : num_patients,
        'num_providers': num_providers, 
        'provider_capacity'    : provider_capacity,
        'top_choice_prob': top_choice_prob, 
        'choice_model': choice_model,
        'exit_option': exit_option,
        'num_trials': num_trials,
        'context_dim': context_dim, 
        'true_top_choice_prob': true_top_choice_prob, 
        'num_repetitions': num_repetitions, 
        'max_menu_size': max_menu_size, 
        'utility_function': utility_function, 
        'order': order, 
        'previous_patients_per_provider': previous_patients_per_provider} 

## Baselines

In [62]:
seed_list = [43,44,45]
restrict_resources()

In [63]:
policy = random_policy
name = "random"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_repetitions*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_repetitions*num_trials*len(seed_list))

random policy
Utility by provider [[6.04884427e-03 0.00000000e+00 0.00000000e+00 3.80189117e-02
  1.38330595e-02 2.75446106e-03 2.35834804e-02 2.44378560e-02
  1.70719057e-02 7.16814307e-03 0.00000000e+00 4.92280109e-03
  1.61051335e-02 5.86712793e-02 4.38067654e-02 7.04447060e-02
  1.02871387e-02 4.91276216e-03 0.00000000e+00 7.00000000e-02
  4.88649135e-02 1.81809316e-02 7.27006117e-03 2.58422818e-02
  2.75484036e-02]
 [5.65431570e-03 5.16389909e-02 2.37056519e-03 2.14263563e-02
  1.95651452e-02 0.00000000e+00 0.00000000e+00 2.89594060e-02
  9.22372741e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.04500650e-02 0.00000000e+00 2.22465122e-02 7.32721041e-02
  1.23925967e-02 1.68937699e-02 1.55158260e-04 3.00000000e-02
  1.10371061e-03 1.29611293e-02 8.19332470e-02 2.14911653e-02
  4.58391995e-02]
 [1.24623840e-02 4.01907325e-02 5.57875521e-03 2.21353406e-03
  7.22643204e-03 2.82319618e-03 8.94760036e-03 1.05967745e-01
  2.54175632e-02 2.29262292e-02 1.67107201e-02 2.12107185e-03


(0.9653333333333334, 0.5802574056759738)

In [64]:
policy = all_ones_policy
name = "greedy_basic"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.mean(results['{}_minimums_all'.format(name)]),np.mean(results['{}_gaps_all'.format(name)]),np.mean(results['{}_variance_all'.format(name)])

greedy_basic policy
Utility by provider [[0.00000000e+00 0.00000000e+00 6.90668825e-04 7.98397145e-02
  8.29983569e-03 0.00000000e+00 5.89587009e-03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.91141862e-02 4.38067654e-02 8.60990852e-02
  0.00000000e+00 2.45638108e-03 0.00000000e+00 8.00000000e-02
  7.32973702e-02 3.63618631e-02 0.00000000e+00 7.10662750e-02
  6.42796085e-02]
 [0.00000000e+00 5.59422401e-02 3.55584779e-03 2.99968988e-02
  6.26084647e-02 0.00000000e+00 0.00000000e+00 2.89594060e-02
  1.29132184e-01 7.79208932e-03 0.00000000e+00 0.00000000e+00
  1.15571614e-02 0.00000000e+00 0.00000000e+00 5.69894143e-02
  0.00000000e+00 1.47820486e-02 0.00000000e+00 4.00000000e-02
  1.10371061e-03 0.00000000e+00 1.27451718e-01 1.07455827e-02
  4.58391995e-02]
 [1.24623840e-02 7.14501911e-02 3.34725313e-03 0.00000000e+00
  5.41982403e-03 2.82319618e-03 0.00000000e+00 1.38573205e-01
  0.00000000e+00 0.00000000e+00 1.67107201e-02 2.1210718

(0.07069784181482378, 0.9275719841160077, 0.0838943347985713)

In [65]:
policy = greedy_policy
name = "greedy"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'])

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

greedy policy
Utility by provider [[0.00000000e+00 0.00000000e+00 6.90668825e-04 7.98397145e-02
  8.29983569e-03 0.00000000e+00 5.89587009e-03 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 3.91141862e-02 4.38067654e-02 8.60990852e-02
  0.00000000e+00 2.45638108e-03 0.00000000e+00 8.00000000e-02
  7.32973702e-02 3.63618631e-02 0.00000000e+00 7.10662750e-02
  6.42796085e-02]
 [0.00000000e+00 5.59422401e-02 3.55584779e-03 2.99968988e-02
  6.26084647e-02 0.00000000e+00 0.00000000e+00 2.89594060e-02
  1.29132184e-01 7.79208932e-03 0.00000000e+00 0.00000000e+00
  1.15571614e-02 0.00000000e+00 0.00000000e+00 5.69894143e-02
  0.00000000e+00 1.47820486e-02 0.00000000e+00 4.00000000e-02
  1.10371061e-03 0.00000000e+00 1.27451718e-01 1.07455827e-02
  4.58391995e-02]
 [1.24623840e-02 7.14501911e-02 3.34725313e-03 0.00000000e+00
  5.41982403e-03 2.82319618e-03 0.00000000e+00 1.38573205e-01
  0.00000000e+00 0.00000000e+00 1.67107201e-02 2.12107185e-03


(1.0, 0.6169282324379038)

In [236]:
if 2**(num_patients*num_providers)*2**(num_patients)*math.factorial(num_patients) < 100000:
    policy = one_shot_policy
    per_epoch_function = optimal_policy
    name = "optimal"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    results['{}_minimums'.format(name)] = rewards['provider_minimums']
    results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
    results['{}_gaps'.format(name)] = rewards['provider_gaps']
    results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
    results['{}_variance'.format(name)] = rewards['provider_variance']
    results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
    results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [237]:
policy = one_shot_policy
per_epoch_function = optimal_order_policy
name = "optimal_order"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

optimal_order policy
Took 0.017654895782470703 time
0.758 0.5405452364134852


## Offline

In [66]:
policy = one_shot_policy
per_epoch_function = lp_policy
name = "lp"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0))

lp policy
Utility by provider [[0.         0.         0.         0.38018912 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.3913029  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.51917553 0.
  0.         0.         0.  

(1.0, 0.6971026994118463, 7.550385695894201)

In [15]:
policy = one_shot_policy
per_epoch_function = lp_workload_policy
name = "lp_workload"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0)),np.max(np.mean(np.array(rewards['final_workloads'])[0],axis=0))

lp_workload policy
Took 0.5431537628173828 time


(0.0, 0.0, 6.892960645691221, 6.892960645691221)

In [21]:
policy = one_shot_policy
per_epoch_function = lp_multiple_match_policy
name = "lp_multiple_match"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_multiple_match policy
Took 0.09562230110168457 time


(0.2536, 0.23916547111746667)

In [16]:
if choice_model == 'threshold':
    policy = one_shot_policy 
    per_epoch_function = lp_threshold
    name = "lp_threshold"
    print("{} policy".format(name))

    rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

    results['{}_matches'.format(name)] = rewards['matches']
    results['{}_utilities'.format(name)] = rewards['patient_utilities']
    results['{}_workloads'.format(name)] = rewards['provider_workloads']

    results['{}_minimums'.format(name)] = rewards['provider_minimums']
    results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
    results['{}_gaps'.format(name)] = rewards['provider_gaps']
    results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
    results['{}_variance'.format(name)] = rewards['provider_variance']
    results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
    results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

    print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [17]:
policy = one_shot_policy 
per_epoch_function = lp_more_patients_policy
name = "lp_more_patients"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_more_patients policy
Took 0.6590728759765625 time


(0.7636, 0.5261014333470473)

In [18]:
policy = one_shot_policy
per_epoch_function = lp_fairness_policy
name = "lp_fairness"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

lp_fairness policy
Took 0.5672309398651123 time


(0.7636, 0.4479570155876951)

In [19]:
policy = one_shot_policy
per_epoch_function = group_based_policy
name = "group_based"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

group_based policy
Took 0.715339183807373 time


(0.7636, 0.5261014333470473)

In [20]:
policy = one_shot_policy
per_epoch_function = group_based_unidirectional_policy
name = "group_based_unidirectional"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

group_based_unidirectional policy
Took 0.7381973266601562 time


(0.7504, 0.5187739882075151)

In [22]:
policy = one_shot_policy 
per_epoch_function = provider_focused_policy
name = "provider_focused"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

provider_focused policy
B 3, Per provider [3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3.
 3.], Per Patient [1. 4. 1. 3. 4. 4. 3. 4. 2. 5. 1. 1. 2. 5. 2. 2. 3. 2. 4. 4. 5. 5. 1. 4.
 3.]
Took 0.1559138298034668 time


(0.6824, 0.511133744634547)

In [7]:
policy = one_shot_policy 
per_epoch_function = provider_focused_less_interference_policy
name = "provider_focused_less_interference"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list))

provider_focused_less_interference policy


NameError: name 'seed_list' is not defined

In [71]:
# policy = one_shot_policy 
# for lamb in [0.25,0.5,1,2,4]:
#     per_epoch_function = provider_focused_linear_regularization_policy(lamb)
#     name = "provider_focused_linear_regularization_{}".format(lamb)
#     print("{} policy".format(name))

#     rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

#     results['{}_matches'.format(name)] = rewards['matches']
#     results['{}_utilities'.format(name)] = rewards['patient_utilities']
#     results['{}_workloads'.format(name)] = rewards['provider_workloads']

#     results['{}_minimums'.format(name)] = rewards['provider_minimums']
#     results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
#     results['{}_gaps'.format(name)] = rewards['provider_gaps']
#     results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
#     results['{}_variance'.format(name)] = rewards['provider_variance']
#     results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
#     results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

#     print(lamb,np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

provider_focused_linear_regularization_0.25 policy
B 1, Per provider [1. 1. 1. 1. 1. 1. 1. 0. 1. 1.], Per Patient [2. 2. 1. 1. 1. 0. 1. 0. 0. 1.]
Took 0.03953838348388672 time
0.25 0.7 0.49648776357158675
provider_focused_linear_regularization_0.5 policy
B 1, Per provider [1. 0. 1. 1. 1. 1. 1. 0. 1. 1.], Per Patient [1. 2. 1. 1. 1. 0. 1. 0. 0. 1.]


/usr0/home/naveenr/projects/patient_provider/patient/provider_policies.py:77: RuntimeWarning: invalid value encountered in double_scalars
  real_value += upper/lower*prod


Took 0.0397343635559082 time
0.5 0.7 0.49648776357158675
provider_focused_linear_regularization_1 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.031838178634643555 time
1 0.0 0.0
provider_focused_linear_regularization_2 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.03140616416931152 time
2 0.0 0.0
provider_focused_linear_regularization_4 policy
B 1, Per provider [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], Per Patient [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Took 0.03318023681640625 time
4 0.0 0.0


In [14]:
# policy = one_shot_policy 
# for lamb in [0,0.1,0.25,0.5]:#,1,2,4]:
#     per_epoch_function = provider_focused_log_regularization_policy(lamb)
#     name = "provider_focused_log_regularization_{}".format(lamb)
#     print("{} policy".format(name))

#     rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

#     results['{}_matches'.format(name)] = rewards['matches']
#     results['{}_utilities'.format(name)] = rewards['patient_utilities']
#     results['{}_workloads'.format(name)] = rewards['provider_workloads']

#     results['{}_minimums'.format(name)] = rewards['provider_minimums']
#     results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
#     results['{}_gaps'.format(name)] = rewards['provider_gaps']
#     results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
#     results['{}_variance'.format(name)] = rewards['provider_variance']
#     results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
#     results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

#     print(lamb,np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [13]:
# policy = one_shot_policy 
# per_epoch_function = gradient_descent_policy
# name = "gradient_descent"
# print("{} policy".format(name))

# rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

# results['{}_matches'.format(name)] = rewards['matches']
# results['{}_utilities'.format(name)] = rewards['patient_utilities']
# results['{}_workloads'.format(name)] = rewards['provider_workloads']

# results['{}_minimums'.format(name)] = rewards['provider_minimums']
# results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
# results['{}_gaps'.format(name)] = rewards['provider_gaps']
# results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
# results['{}_variance'.format(name)] = rewards['provider_variance']
# results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
# results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

# print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

In [67]:
policy = one_shot_policy 
per_epoch_function = gradient_descent_policy_2
name = "gradient_descent_2"
print("{} policy".format(name))

rewards, simulator = run_multi_seed(seed_list,policy,results['parameters'],per_epoch_function)

results['{}_matches'.format(name)] = rewards['matches']
results['{}_utilities'.format(name)] = rewards['patient_utilities']
results['{}_workloads'.format(name)] = rewards['provider_workloads']

results['{}_minimums'.format(name)] = rewards['provider_minimums']
results['{}_minimums_all'.format(name)] = rewards['provider_minimums_all']
results['{}_gaps'.format(name)] = rewards['provider_gaps']
results['{}_gaps_all'.format(name)] = rewards['provider_gaps_all']
results['{}_variance'.format(name)] = rewards['provider_variance']
results['{}_variance_all'.format(name)] = rewards['provider_variance_all']
results['{}_workload_diff'.format(name)] = [max(rewards['final_workloads'][0][i])-max(rewards['initial_workloads'][0][i]) for i in range(len(rewards['final_workloads'][0]))]

print(np.sum(rewards['matches'])/(num_patients*num_trials*len(seed_list)),np.sum(rewards['patient_utilities'])/(num_patients*num_trials*len(seed_list)))

gradient_descent_2 policy
Utility by provider [[0.         0.         0.         0.38018912 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.3913029  0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         1.         0.         0.         0.         0.
  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.51917553 0.
  0.        

In [37]:

def objective2(z, theta, p, sorted_theta,lamb=1, smooth_reg='entropy', epsilon=1e-5):
    # Reparameterize x using sigmoid
    x = torch.sigmoid(z)  # x is now bounded in [0, 1]
    
    # Compute the sum of x over rows for each column
    sum_x = torch.sum(x, dim=0)  # Shape: (columns,)
    # Compute the sum of x across all columns for each row
    row_sums = torch.sum(x, dim=1, keepdim=True)  # Shape: (rows, 1)
    
    # Normalize x by row sums
    normalized_x = x / (p*torch.maximum(row_sums, torch.tensor(1.0, device=sum_x.device)))*(1-(1-p)**(torch.maximum(row_sums, torch.tensor(1.0, device=sum_x.device)))) 

    sorted_normalized_x = normalized_x.gather(1, sorted_theta)

    # Compute cumulative products (1 - normalized_x) along rows
    one_minus_sorted = 1 - sorted_normalized_x
    cumprods = torch.cumprod(one_minus_sorted, dim=1)

    # Shift the cumulative products to use for the original scaling (prepending 1 for first index)
    shifted_cumprods = torch.cat([torch.ones(cumprods.size(0), 1, device=cumprods.device), cumprods[:, :-1]], dim=1)

    # Apply the cumulative product scaling to the original indices
    scaled_normalized_x = sorted_normalized_x * shifted_cumprods

    # Scatter back to the original positions
    normalized_x = torch.zeros_like(normalized_x)
    normalized_x.scatter_(1, sorted_theta, scaled_normalized_x)
    # Normalize row-wise

    normalized_x /= (torch.sum(normalized_x, dim=1, keepdim=True) + 1e-8)
    prod = (1 - (1 - p) ** torch.sum(normalized_x,dim=0)) 

    # Compute numerator for the first term (using normalized x)
    term1_num = prod * torch.sum(normalized_x * theta, dim=0)

    term1_den = torch.sum(normalized_x, dim=0) + 1e-8  # Avoid division by zero
    term1_den = torch.maximum(term1_den,torch.tensor(1.0, device=sum_x.device))

    # Compute the main term
    term1 = (term1_num / term1_den)
        
    term1 = torch.sum(term1) / theta.shape[1]  # Normalize by number of columns

    reg_term = 0
    # Add smooth regularization term
    if smooth_reg == 'logit' and lamb > 0:
        reg_term = torch.sum(torch.logit(x, eps=epsilon) ** 2)  # Logit-based penalty
    elif smooth_reg == 'entropy' and lamb > 0:
        reg_term = -torch.sum(x * torch.log(x + epsilon) + (1 - x) * torch.log(1 - x + epsilon))  # Entropy-based penalty
    loss = term1 - lamb * reg_term

    return loss


In [38]:
objective2(ones_tensor*10000-10000/2,theta,p,sorted_theta,0)

tensor(0.0359)

In [36]:
x

tensor([[0., 1.],
        [0., 1.]])

In [39]:
if is_jupyter:
    theta = [p.provider_rewards for p in simulator.patients]
    theta = torch.Tensor(theta)
    sorted_theta = torch.argsort(theta, dim=1,descending=True)  # Sorting indices of `theta` row-wise
    opt_tensor = torch.Tensor(lp_policy(simulator))
    ones_tensor = torch.Tensor(np.ones(opt_tensor.shape))
    x = torch.Tensor(gradient_descent_policy_2(simulator))
    p = true_top_choice_prob
    print(objective2(opt_tensor*10000-10000/2,theta,p,sorted_theta,0),objective2(ones_tensor*10000-10000/2,theta,p,sorted_theta,0),objective2(x*10000-10000/2,theta,p,sorted_theta,0))
    print(objective(opt_tensor*10000-10000/2,theta,p,sorted_theta,0),objective(ones_tensor*10000-10000/2,theta,p,sorted_theta,0),objective(x*10000-10000/2,theta,p,sorted_theta,0))

tensor(0.0396) tensor(0.0359) tensor(0.0250)
tensor(0.0396) tensor(0.0456) tensor(0.0475)


## Save Data

In [ ]:
save_path = get_save_path(out_folder,save_name)

In [ ]:
delete_duplicate_results(out_folder,"",results)

In [ ]:
json.dump(results,open('../../results/'+save_path,'w'),cls=MyEncoder)